## Load Module Files from GitHub

In [1]:
!rm /content/*.py*

rm: cannot remove '/content/*.py*': No such file or directory


In [2]:
queries = []

filenames = [
             "Conv2DMHAUnit.py",
             "Conv2DmhaLSTMCell.py",
             "conv2d_mha_utils.py"
]

for fn in filenames:
  queries.append(f"https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/Conv2DAttentionLSTM/{fn}")
queries.append("https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/imports.py")

In [3]:
queries

['https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/Conv2DAttentionLSTM/Conv2DMHAUnit.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/Conv2DAttentionLSTM/Conv2DmhaLSTMCell.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/Conv2DAttentionLSTM/conv2d_mha_utils.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/imports.py']

In [4]:
import subprocess
# Download and import my module files from GitHub
result = subprocess.run(["wget", "--no-cache", "--backups=1"] + queries, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
print(result.stderr.decode("utf-8"))

--2022-07-28 16:34:54--  https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/Conv2DAttentionLSTM/Conv2DMHAUnit.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3980 (3.9K) [text/plain]
Failed to rename Conv2DMHAUnit.py to Conv2DMHAUnit.py.1: (2) No such file or directory
Saving to: ‘Conv2DMHAUnit.py’

     0K ...                                                   100% 28.0M=0s

2022-07-28 16:34:54 (28.0 MB/s) - ‘Conv2DMHAUnit.py’ saved [3980/3980]

--2022-07-28 16:34:54--  https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/Conv2DAttentionLSTM/Conv2DmhaLSTMCell.py
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 2628 (2.6K)

## Load and Create Google Drive Linked Environment

In [5]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
unscaled_maps_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/covid_map_data/infection_maps_unscaled.npy"
unscaled_metadata_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/covid_map_data/infection_unscaled_metadata.json"
scaled_maps_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/covid_map_data/infection_maps_scaled.npy"
scaled_metadata_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/covid_map_data/infection_scaled_metadata.json"
dataset_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/US_time_series_covid_cases.csv"

In [7]:
from conv2d_mha_utils import load_sequential_data, create_flow, create_conv_mha_lstm_model, tf

# Unscaled Maps

## Load Sequential Data from Google Drive

In [8]:
BATCH_SIZE = 16

EPOCHS = 10
IMAGE_X, IMAGE_Y = 64, 64
NUM_DAYS_PER_SAMPLE = 7

In [9]:
X_indices, y_infection, y_death, X_maps = load_sequential_data(
    maps_path = unscaled_maps_path,
    metadata_path = unscaled_metadata_path,
    dataset_path = dataset_path,
    image_x = IMAGE_X,
    image_y = IMAGE_Y,
    num_days_per_sample = NUM_DAYS_PER_SAMPLE
)

Loading Image Indices...


41993it [00:01, 22138.77it/s]


## Split Train Test

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_indices, y_infection, train_size = .8)

## Create LSTM Model

In [12]:
MODEL_ARCHITECTURE = (16, 16, 16)
KERNEL_SIZE = (4, 4)
OUTPUT_SIZE = 49

In [13]:
seq = tf.keras.models.Sequential()
seq.add(tf.keras.layers.Input(shape = (NUM_DAYS_PER_SAMPLE, IMAGE_X, IMAGE_Y, 1)))
for i in range(len(MODEL_ARCHITECTURE) - 1):
  seq.add(tf.keras.layers.ConvLSTM2D(MODEL_ARCHITECTURE[i], KERNEL_SIZE, padding = "same", return_sequences = True))
  seq.add(tf.keras.layers.MaxPool3D((1,4,4)))
  seq.add(tf.keras.layers.LeakyReLU())
  seq.add(tf.keras.layers.LayerNormalization())
seq.add(tf.keras.layers.ConvLSTM2D(MODEL_ARCHITECTURE[-1], KERNEL_SIZE, padding = "same", return_sequences = False))
seq.add(tf.keras.layers.LeakyReLU())
seq.add(tf.keras.layers.LayerNormalization())
seq.add(tf.keras.layers.Flatten())
seq.add(tf.keras.layers.Dense(OUTPUT_SIZE, activation = "linear"))

In [14]:
seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 7, 64, 64, 16)     17472     
                                                                 
 max_pooling3d (MaxPooling3D  (None, 7, 16, 16, 16)    0         
 )                                                               
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 7, 16, 16, 16)     0         
                                                                 
 layer_normalization (LayerN  (None, 7, 16, 16, 16)    32        
 ormalization)                                                   
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 7, 16, 16, 16)     32832     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 7, 4, 4, 16)      0

## Train Model

In [15]:
optimizer = tf.keras.optimizers.Adam()
BATCH_SIZE = 16
EPOCHS = 20
STEPS_PER_EPOCH = X_train.shape[0] // BATCH_SIZE

In [16]:
@tf.function
def loss_func(y_true, y_pred):
  squared_diff = tf.square(y_true - y_pred)
  mse = tf.reduce_mean(squared_diff, axis=-1)
  squared_diff_mean = tf.square(tf.reduce_mean(y_true, axis = -1) - tf.reduce_mean(y_pred, axis = -1))
  return mse + squared_diff_mean

In [17]:
seq.compile(
    loss = loss_func,
    optimizer = optimizer,
    metrics = ["mse", "mae"]
)

In [18]:
train_generator = create_flow(X_train, y_train, BATCH_SIZE, X_maps)

In [19]:
seq.fit(
    x = train_generator,
    batch_size = BATCH_SIZE,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS)

Epoch 1/20
42/42 [==============================] - 23s 162ms/step - loss: 0.0629 - mse: 0.0622 - mae: 0.1307
Epoch 2/20
42/42 [==============================] - 7s 161ms/step - loss: 2.8865e-04 - mse: 2.8449e-04 - mae: 0.0118
Epoch 3/20
42/42 [==============================] - 7s 156ms/step - loss: 2.8975e-05 - mse: 2.8595e-05 - mae: 0.0040
Epoch 4/20
42/42 [==============================] - 7s 159ms/step - loss: 2.0686e-05 - mse: 2.0257e-05 - mae: 0.0034
Epoch 5/20
42/42 [==============================] - 7s 159ms/step - loss: 1.7596e-05 - mse: 1.7274e-05 - mae: 0.0032
Epoch 6/20
42/42 [==============================] - 7s 156ms/step - loss: 1.4413e-05 - mse: 1.4042e-05 - mae: 0.0029
Epoch 7/20
42/42 [==============================] - 7s 160ms/step - loss: 1.1949e-05 - mse: 1.1587e-05 - mae: 0.0026
Epoch 8/20
42/42 [==============================] - 7s 160ms/step - loss: 1.0582e-05 - mse: 1.0326e-05 - mae: 0.0025
Epoch 9/20
42/42 [==============================] - 7s 156ms/step - los

In [20]:
test_generator = create_flow(X_test, y_test, BATCH_SIZE, X_maps)

In [21]:
STEPS = X_test.shape[0] // BATCH_SIZE

In [22]:
eval_res = seq.evaluate(
    x = test_generator,
    steps = STEPS,
    batch_size = BATCH_SIZE)

10/10 [==============================] - 1s 29ms/step - loss: 1.3616e-05 - mse: 1.1283e-05 - mae: 0.0027


In [23]:
eval_res

[1.3616465366794728e-05, 1.1282940249657258e-05, 0.002709501888602972]